In [17]:
import glob
import os
os.chdir("/Users/hbg/Documents/filtered_contigs/KV")
f1 = glob.glob("best_hit/Megahit_assembled/*")

In [18]:
f1[1]

'best_hit/Megahit_assembled/SRR14888015.txt'

In [19]:
file_name = [os.path.basename(f) for f in f1]
SRAs = [f.replace(".txt", "") for f in file_name]

In [20]:
SRAs[1]

'SRR14888015'

In [21]:
import pandas as pd
data = pd.read_csv("KV.csv")
acc = list(data.acc)

In [22]:
import pycountry_convert as pc

def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name


In [23]:
country_name = 'Germany'
print(country_to_continent(country_name))

Europe


In [24]:
acc.index(SRAs[1])
data.organism[0],data.geo_loc_name_country_calc[0],data.collection_date_sam[0]

('Varroa destructor', 'China', nan)

In [25]:
type(data.geo_loc_name_country_calc[23])

str

In [26]:
import csv
import re
total = 0
count = 0
filenames = []
country_names =[]
location = []
os.mkdir('filtered_with_host_names/')

for i in range(0,len(f1)):
    try:
        total+=1
        with open(f1[i]) as f:
            reader = csv.reader(f, delimiter="\n")
            d = list(reader)
            k = d[0][0].split()
        if (float(k[1]) > 80 and float(k[2]) > 5000 and float(k[3]) == 0.0 and float(k[4]) > 6000):
            count+=1
            ind = acc.index(SRAs[i])

            if (type(data.collection_date_sam[ind])==str):
                year = data.collection_date_sam[ind][1:-1]
            else:
                year = '2018-01-01'
            if (type(data.geo_loc_name_country_calc[ind])==str and data.geo_loc_name_country_calc[ind] != 'uncalculated'):
                country = data.geo_loc_name_country_calc[ind]
                country_names.append(country)
                location.append(country_to_continent(country))
            else:
                country = 'Unknown'
                country_names.append(country)
                location.append("?")

            name = data.organism[ind] + '_' + country + '_@' + year + '@' + str(i)
            name = re.sub(" ","_",name)
            filenames.append(name)
            with open("filtered_with_host_names/"+name+".fa", 'w') as f:
                writer = csv.writer(f)
                writer.writerow(d[1:][0])
                writer.writerow(d[2:][0])
    except:
        total+=1

In [27]:
count,total

(170, 321)

In [28]:
with open('host_names.txt', 'w') as f:
    for line in filenames:
        f.write("%s\n" % line)

In [29]:
df = pd.DataFrame(list(zip(filenames, country_names, location)), columns =['traits', 'country', 'continents']) 
df.to_csv('host_names_with_locations.txt', sep='\t', encoding='utf-8')
print(df) 


                                          traits      country continents
0    Varroa_destructor_New_Zealand_@2018-01-01@0  New Zealand    Oceania
1           Apis_mellifera_Unknown_@2018-01-01@1      Unknown          ?
2       Apis_mellifera_South_Korea_@2018-08-22@7  South Korea       Asia
3           Apis_mellifera_Unknown_@2018-01-01@9      Unknown          ?
4         Varroa_destructor_China_@2018-01-01@14        China       Asia
..                                           ...          ...        ...
165    Varroa_destructor_Unknown_@2018-01-01@313      Unknown          ?
166     Varroa_destructor_Poland_@2000-04-12@314       Poland     Europe
167    Varroa_destructor_Unknown_@2018-01-01@315      Unknown          ?
168   Apis_mellifera_South_Korea_@2018-08-22@317  South Korea       Asia
169        Apis_mellifera_Turkey_@2021-07-01@320       Turkey       Asia

[170 rows x 3 columns]
